In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier, plot_tree
import shap
import datetime

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pd.options.display.max_columns = None

sns.set(font_scale=1.4)
sns.set_style({'font.family': 'serif',
               'fontname': 'Times New Roman'})

In [ ]:
datatran_df = pd.read_csv('../input/brazil-highway-traffic-accidents/por_ocorrencias/datatran2017.csv', 
                          sep = ';',
                          encoding = 'latin-1')

In [ ]:
datatran_model_data_df = datatran_df.copy()

In [ ]:
datatran_model_data_df['uop'].fillna('UOP-NI', inplace=True)

In [ ]:
mean_br = datatran_model_data_df['br'].mode().mean();
datatran_model_data_df['br'].fillna(mean_br, inplace=True);

In [ ]:
datatran_model_data_df['km'] = datatran_model_data_df['km'].astype(str).str.replace(',','.').astype(float)
mean_km = datatran_model_data_df['km'].mean();
datatran_model_data_df['km'].fillna(mean_km, inplace=True);

In [ ]:
datatran_model_data_df['horario'] = [(datetime.datetime.strptime(hora, "%H:%M:%S")- datetime.datetime(1900, 1, 1)).total_seconds() for hora in datatran_model_data_df['horario']];

In [ ]:
datatran_model_data_df['latitude'] = datatran_model_data_df['latitude'].astype(str).str.replace(',','.').astype(float)
datatran_model_data_df['longitude'] = datatran_model_data_df['longitude'].astype(str).str.replace(',','.').astype(float)

In [ ]:
datatran_model_data_df.dtypes

In [ ]:
plt.figure(figsize = (30,8))

sns.heatmap(datatran_model_data_df.isnull(),
            yticklabels = 0,
            cbar = False)

plt.title('Visão geral dos valores faltantes')
plt.xlabel('Colunas')
plt.show()

In [ ]:
datatran_model_data_df.drop(['id', 'data_inversa', 'classificacao_acidente', 'fase_dia'],
                            axis='columns', inplace=True)

In [ ]:
colunas = list(datatran_model_data_df.columns.values) 
colunas.pop(colunas.index('uso_solo'))
datatran_model_data_df = datatran_model_data_df[colunas + ['uso_solo']]

In [ ]:
datatran_model_data_df.head()

In [ ]:
##Label Encoding

cleanup_nums = {"sentido_via":{"Crescente": 1, "Não Informado": 0, "Decrescente": -1},
                "tipo_pista": {"Simples": 0, "Dupla":1, "Múltipla":2},
               "tracado_via":{"Desvio Temporário":"Curva", "Viaduto":"Ponte", "Retorno Regulamentado":"Interseção de vias"}}